In [ ]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

import scipy.stats as stats
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle 

import sklearn
from sklearn import metrics

import alibi 
from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.saving import save_detector, load_detector

import sqlalchemy
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import *

import xgboost
from datetime import datetime, timedelta
import time  
import pytz    
tz_NY = pytz.timezone('Asia/Kolkata')

import snowflake_creds

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="cr21746.ap-south-1",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

## Creating the Model and Data drift detector object from Training set:

#### Data Drift detector:

In [ ]:
query = """

SELECT CASE_ID,
           COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
           COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
           COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
           COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
           COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
           COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
           COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
           COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
           COALESCE(BED_GRADE,0) AS BED_GRADE,
           PATIENTID,
           COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
           COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
           COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
           COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
           COALESCE(AGE,'None') AS AGE,
           COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
           ADMISSION_DATE,
           DISCHARGE_DATE

    FROM HEALTHDB.HEALTHSCHEMA.HEALTH_DATA

"""

In [ ]:
# Loading the train data
with engine.connect() as conn:
    df_train = pd.DataFrame(pd.read_sql(query,conn))
    df_train.columns = [col.upper() for col in df_train.columns.tolist()]

In [ ]:
print(df_train.shape)
df_train.head(3)

In [ ]:
df_train.info()

In [ ]:
# Getting the numerical and categorical columns for creating the datadrift object
num_columns = ['AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL','VISITORS_WITH_PATIENT','ADMISSION_DEPOSIT']
id_columns = ['CASE_ID','PATIENTID','ADMISSION_DATE','DISCHARGE_DATE']
cat_columns = [col for col in df_train.columns.tolist() if col not in num_columns+id_columns]

In [ ]:
num_columns

In [ ]:
cat_columns

In [ ]:
X_train = df_train[num_columns + cat_columns]
print(X_train.shape)
X_train.head()

In [ ]:
cat_indices = np.arange(3,15)
cat_indices

In [ ]:
# category dict for the driftdetector to identify unique categories
categories_per_feature = {f: None for f in cat_indices}
categories_per_feature

In [ ]:
# Initialize the detector
cd = TabularDrift(X_train.values, p_val=.05, categories_per_feature=categories_per_feature)

In [ ]:
cd

In [ ]:
cd.get_config()

In [ ]:
# # Using pickle to save and load it the trained detector
# with open('Trained_Drift_Detector.pkl','wb') as F:
#     pickle.dump(cd,F)

with open('Trained_Drift_Detector.pkl','rb') as F:
    trained_drift_model = pickle.load(F)    

In [ ]:
trained_drift_model.get_config()

In [ ]:
preds = trained_drift_model.predict(X_train.values)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

In [ ]:
# If you are interested in individual feature-wise drift, this is also possible:
fpreds = trained_drift_model.predict(X_train.values, drift_type='feature')
fpreds

In [ ]:
for f in range(trained_drift_model.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    # print(f, stat)
    fname = X_train.columns.tolist()[f]
    # print(f, fname)
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

In [ ]:
temp = pd.DataFrame()
temp['Time Period'] = [str('2023-01-01') + ' to ' + str('2023-01-07')]*len(X_train.columns.tolist())
temp['Features'] = X_train.columns.tolist()
temp['Is Drift'] = fpreds['data']['is_drift']
temp['Stat Test'] = temp['Features'].apply(lambda x: 'Chi2' if x in cat_columns else 'K-S')
temp['Stats Value'] = fpreds['data']['distance']
temp['P-value'] = fpreds['data']['p_val']
print(temp.shape)
temp

##### Creating a noise in the train data to check if drift detector is working

In [ ]:
temp = X_train.copy()
temp.loc[:5,'HOSPITAL_CODE'] = 100
temp.head()

In [ ]:
# If you are interested in individual feature-wise drift, this is also possible:
fpreds = trained_drift_model.predict(temp.values, drift_type='feature')
fpreds

In [ ]:
for f in range(trained_drift_model.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    # print(f, stat)
    fname = temp.columns.tolist()[f]
    # print(f, fname)
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

#### Data Drift Scoring:

In [ ]:
def data_monitoring_batch_query(a):
    query = f"""

        SELECT CASE_ID,
               COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
               COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
               COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
               COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
               COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
               COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
               COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
               COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
               COALESCE(BED_GRADE,0) AS BED_GRADE,
               PATIENTID,
               COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
               COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
               COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
               COALESCE(VISITORS_WITH_PATIENT_X,0) AS VISITORS_WITH_PATIENT,
               COALESCE(AGE,'None') AS AGE,
               COALESCE(ADMISSION_DEPOSIT_X,0) AS ADMISSION_DEPOSIT,
               ADMISSION_DATE,
               DISCHARGE_DATE

            FROM HEALTHDB.HEALTHSCHEMA.TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
            WHERE ADMISSION_DATE >= CURRENT_DATE-144+{a*7} AND ADMISSION_DATE < CURRENT_DATE-144+{(a+1)*7}        

        """
    return query

In [ ]:
def data_monitoring(batch_id):
    # Loading the train data
    with engine.connect() as conn:
        batch_df = pd.DataFrame(pd.read_sql(data_monitoring_batch_query(batch_id),conn))
        batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]
    
    # Getting the numerical and categorical columns for creating the datadrift object
    num_columns = ['AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL','VISITORS_WITH_PATIENT','ADMISSION_DEPOSIT']
    id_columns = ['CASE_ID','PATIENTID','ADMISSION_DATE','DISCHARGE_DATE']
    cat_columns = [col for col in batch_df.columns.tolist() if col not in num_columns+id_columns]
    
    # Getting the final prepared data
    batch_final = batch_df[num_columns + cat_columns]
    
    # Loading the Trained data drift detector
    with open('Trained_Drift_Detector.pkl','rb') as F:
        trained_drift_model = pickle.load(F)    
    
    # Checking for drift
    # If you are interested in individual feature-wise drift, this is also possible:
    fpreds = trained_drift_model.predict(batch_final.values, drift_type='feature')
    
    log_df = pd.DataFrame()
    log_df['Time Period'] = ([str(batch_df['ADMISSION_DATE'].min()) + ' to ' + 
                              str(batch_df['ADMISSION_DATE'].max())]
                              * len(batch_final.columns.tolist())
                            )
    log_df['Total Records'] = batch_df.shape[0]
    log_df['Features'] = batch_final.columns.tolist()
    log_df['Is Drift'] = fpreds['data']['is_drift']
    log_df['Stat Test'] = log_df['Features'].apply(lambda x: 'Chi2' if x in cat_columns else 'K-S')
    log_df['Stats Value'] = np.round(fpreds['data']['distance'])
    log_df['P-value'] = np.round(fpreds['data']['p_val'])
    
    return log_df

In [ ]:
data_monitoring(1)

In [ ]:
t = data_monitoring(0)
t

In [ ]:
print(t.shape[0])
print(t['Is Drift'].sum())

### Model Drift detector:

In [ ]:
# Get the data from the logging table in batches (7 days)
# Using the predicted and actual LOS calculate the performance metrics dict
# Then use the ref_metric_dict (from training) to compare with the current_metric_dict for model drift

In [ ]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="cr21746.ap-south-1",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

In [ ]:
# Function to check model drift
def check_model_drift(ref_metric_dict,cur_metric_dict,type='classification',tol=0.1):
    if type == 'classification':
        precision_change = abs((cur_metric_dict['Precision']-ref_metric_dict['Precision'])/ref_metric_dict['Precision'])
        recall_change = abs((cur_metric_dict['Recall']-ref_metric_dict['Recall'])/ref_metric_dict['Recall'])
        roc_auc_change = abs((cur_metric_dict['Roc-Auc']-ref_metric_dict['Roc-Auc'])/ref_metric_dict['Roc-Auc'])

        counter = 0
        for i in [precision_change,recall_change,roc_auc_change]:
            if i > tol:
                counter += 1

        if counter > 0:
            print("ALERT! There is a model drift.")
            print("Change in Precision: "+ str(np.round(100*precision_change,2))+"%")
            print("Change in Recall: "+ str(np.round(100*recall_change,2))+"%")
            print("Change in Roc-Auc: "+ str(np.round(100*roc_auc_change,2))+"%")
            return 1
        else:
            print("There is no model drift.")
            return 0

    elif type == 'regression':
        rmse_change = abs((cur_metric_dict['RMSE']-ref_metric_dict['RMSE'])/ref_metric_dict['RMSE'])
        mae_change = abs((cur_metric_dict['MAE']-ref_metric_dict['MAE'])/ref_metric_dict['MAE'])
        
        counter = 0
        for i in [rmse_change,mae_change]:
            if i > tol:
                counter += 1

        if counter > 0:
            print("ALERT! There is a model drift.")
            RMSE_CHANGE = np.round(100*rmse_change,2)
            MAE_CHANGE = np.round(100*mae_change,2)
            print("Change in RMSE: "+ str(np.round(100*rmse_change,2))+"%")
            print("Change in MAE: "+ str(np.round(100*mae_change,2))+"%")
            return 1, RMSE_CHANGE, MAE_CHANGE
        else:
            print("There is no model drift.")
            RMSE_CHANGE = 'NONE'
            MAE_CHANGE = 'NONE'
            return 0, RMSE_CHANGE, MAE_CHANGE
        
        

In [ ]:
def model_monitoring_batch_query(a):
    query_sim = f"""

        SELECT *
        FROM TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
        WHERE ADMISSION_DATE >= CURRENT_DATE-144+{a*7} AND ADMISSION_DATE < CURRENT_DATE-144+{(a+1)*7}
        
    """
    return query_sim

In [ ]:
print(model_monitoring_batch_query(2))

In [ ]:
# Loading the train data
with engine.connect() as conn:
    batch_df = pd.DataFrame(pd.read_sql(model_monitoring_batch_query(0),conn))
    batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]

In [ ]:
print(batch_df.shape)
batch_df.head()

In [ ]:
# Creating the current performance dict (from scoring)

actual = batch_df['LOS_X']
predicted = batch_df['PREDICTED_LOS']

rmse = np.sqrt(metrics.mean_squared_error(actual,predicted))
mae = np.sqrt(metrics.mean_absolute_error(actual,predicted))
print("RMSE: ", rmse)
print("MAE: ", mae)

scoring_ref_metrics = {}
scoring_ref_metrics['RMSE'] = rmse
scoring_ref_metrics['MAE'] = mae #+ 0.2*mae
print(scoring_ref_metrics)

In [ ]:
# Loading the reference performance dict (from training)

with open('MODEL_XGB_PERFM_METRICS.pkl', 'rb') as F:
    model_ref_metric = pickle.load(F)

model_ref_metric

In [ ]:
check_model_drift(model_ref_metric,scoring_ref_metrics,type='regression',tol=0.1)

In [ ]:
def model_monitoring(batch_id):
    # Loading the train data
    with engine.connect() as conn:
        batch_df = pd.DataFrame(pd.read_sql(model_monitoring_batch_query(batch_id),conn))
        batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]
    
#     print(batch_df.shape)
    
    # Creating the current performance dict (from scoring)
    actual = batch_df['LOS_X']
    predicted = batch_df['PREDICTED_LOS']

    rmse = np.sqrt(metrics.mean_squared_error(actual,predicted))
    mae = np.sqrt(metrics.mean_absolute_error(actual,predicted))
#     print("RMSE: ", rmse)
#     print("MAE: ", mae)

    scoring_ref_metrics = {}
    scoring_ref_metrics['RMSE'] = rmse
    scoring_ref_metrics['MAE'] = mae #+ 0.2*mae
#     print(scoring_ref_metrics)
    
    
    # Loading the reference performance dict (from training)
    with open('MODEL_XGB_PERFM_METRICS.pkl', 'rb') as F:
        model_ref_metric = pickle.load(F)
        
#     print(model_ref_metric)
    
    # Check for model drift
    model_drift, RMSE_CHANGE, MAE_CHANGE = check_model_drift(model_ref_metric,scoring_ref_metrics,type='regression',tol=0.1)
    
    # Log values
    log = {}
    log['Time Period'] = str(batch_df['ADMISSION_DATE'].min()) + ' to ' + str(batch_df['ADMISSION_DATE'].max())
    log['Total Records'] = batch_df.shape[0]
    log['Scoring Metrics'] = scoring_ref_metrics
    log['Training Metrics'] = model_ref_metric
    log['Model Drift IND'] = model_drift
    log['RMSE Change'] = RMSE_CHANGE
    log['MAE Change'] = MAE_CHANGE
    
    return log
    

In [ ]:
model_monitoring(3)

# Model Monitoring & Retraining Pipeline:

In [ ]:
data_log_df = data_monitoring(0)
model_log_dict = model_monitoring(0)

In [ ]:
data_log_df

In [ ]:
# Data drift condition
data_log_df['Is Drift'].sum() > 0

In [ ]:
model_log_dict

In [ ]:
# Model drift condition
model_log_dict['Model Drift IND']

In [ ]:
# Max date for retraining 
max_date = model_log_dict['Time Period'].split(' ')[2]
max_date